In [30]:
%pip install -qU langchain langchain-community langchain-groq langchain-core langchain-pinecone gpt4all langgraph sentence-transformers gradio langchain-huggingface groq ollama cohere langchain pinecone-client PyPDF2 einops html2text

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install -qU \
    datasets==2.19.1 \
    langchain-pinecone==0.1.1 \
    langchain-openai==0.1.9 \
    langchain==0.2.5 \
    langchain-core==0.2.9 \
    langgraph==0.1.1 \
    semantic-router==0.0.48 \
    serpapi==0.1.5 \
    google-search-results==2.4.2 \
    pygraphviz==1.12  # for visualizing

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [31]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nest_asyncio
from langchain_community.vectorstores.chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
from langchain.chains.combine_documents import stuff
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from IPython.core.display import Markdown
import json
import re
from langchain_core.runnables import (
    RunnableParallel,
    RunnableBranch,
    RunnablePassthrough,
)
from langchain_core.pydantic_v1 import validator
from langchain_core.messages import HumanMessage, AIMessage
from operator import itemgetter
import asyncio
import warnings
import PyPDF2
from typing import overload, Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq
# from sentence_transformers import SentenceTransformer
from langchain.document_transformers import Html2TextTransformer
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from typing import TypedDict, Annotated
from langchain_core.documents import Document
from langgraph.prebuilt import ToolInvocation, ToolExecutor
from langchain_core.tools import Tool
from langchain_core.messages.base import BaseMessage
import operator

warnings.filterwarnings("ignore")

In [32]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path

# Load path from the environment variable
env_ih1 = os.getenv("ENV_IH1")

dotenv_path = Path(env_ih1)
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY= os.getenv('PINECONE_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
STEAMSHIP_API_KEY = os.getenv('STEAMSHIP_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
GEMINI_KEY = os.getenv('GEMINI_KEY')

os.environ['PATH'] += os.pathsep + '/usr/bin'

In [33]:
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(
    model='text-embedding=embedding-3-small',
    openai_api_key=OPENAI_API_KEY,
    # device=device -> Not compatible with OpenAI embeddings
)

vectorstore = PineconeVectorStore(index_name='memory-project', embedding=embedding)

In [45]:
from langchain_openai import OpenAIEmbeddings
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

# Define the embeddings configuration
class Config:
    OPENAI_API_KEY = OPENAI_API_KEY  # Replace with your OpenAI API Key

embedding = OpenAIEmbeddings(
    model="text-embedding-embedding-3-small",
    openai_api_key=Config.OPENAI_API_KEY
)

# Mocking Pinecone Vector Store
class MockVectorStore:
    def __init__(self, index_name, embedding):
        self.index_name = index_name
        self.embedding = embedding

    def similarity_search(self, query, k):
        # Mock search logic (replace this with actual Pinecone interaction)
        return [f"Doc {i+1}: Relevant to {query}" for i in range(k)]

vectorstore = MockVectorStore(index_name="memory-project", embedding=embedding)

# Define the state schema
class QueryState(TypedDict):
    query: str
    embedding_info: Annotated[list, add_messages]
    results: list

# LangGraph StateGraph workflow
memory_saver = MemorySaver()
workflow = StateGraph(state_schema=QueryState)

# Embedding generation node
def generate_embedding(state: QueryState):
    # Logic for embedding generation
    return {"embedding_info": f"Generated embedding for query: {state['query']}"}

# Vector store similarity search node
def perform_similarity_search(state: QueryState):
    query = state["query"]
    k = 3
    results = vectorstore.similarity_search(query, k)
    return {"results": results}

# Adding nodes to the workflow
workflow.add_node("embedding", generate_embedding)
workflow.add_node("similarity_search", perform_similarity_search)

# Connecting nodes
workflow.add_edge(START, "embedding")
workflow.add_edge("embedding", "similarity_search")
workflow.add_edge("similarity_search", END)

# Compile the workflow
compiled_workflow = workflow.compile(checkpointer=memory_saver)

# Example usage with a query
input_state = {"query": "Who is Dreyfus?"}

# Provide a configuration with the required keys
config = {
    "configurable": {
        "thread_id": "example-thread-1",  # Unique identifier for this workflow run
        "checkpoint_ns": "",  # Optional namespace, can be left as an empty string
        "checkpoint_id": None,  # Optional checkpoint ID to resume from, use None for a fresh run
    }
}

# Invoke the workflow
outputs = compiled_workflow.invoke(input_state, config=config)

print("Workflow Outputs:")
print(outputs)

Workflow Outputs:
{'query': 'Who is Dreyfus?', 'embedding_info': [HumanMessage(content='Generated embedding for query: Who is Dreyfus?', additional_kwargs={}, response_metadata={}, id='efef83d2-1ae8-498d-b6c4-eacbcb8e9f85')], 'results': ['Doc 1: Relevant to Who is Dreyfus?', 'Doc 2: Relevant to Who is Dreyfus?', 'Doc 3: Relevant to Who is Dreyfus?']}
